Änderungen, die im Vergleich zum zuerst verwendeten Netzwerk gemacht wurden:

- **Anzahl der Layers**
1. Erste Schicht - 16 -> 32
2. Zweite Schicht - 32 -> 64
3. Dritte Schicht - 64 -> 128
4. Vierte Schicht - neue Schicht mit 256x256 Filter

- **Batch Normalization**

- **Dropout**

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryIoU, Precision, Recall

def unet_model(input_size, dropout_rate=0.2):
    inputs = Input(input_size)
    
    # Encoder with BatchNorm and Dropout
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)  # Increased from 16
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout_rate)(p1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)  # Increased from 32
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout_rate)(p2)
    
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)  # Increased from 64
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout_rate)(p3)
    
    # Deeper encoder for better feature extraction
    c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout_rate)(p4)
    
    # Bottleneck
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(p4)  # Increased from 128
    c5 = BatchNormalization()(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Dropout(dropout_rate * 1.5)(c5)  # Higher dropout in bottleneck
    
    # Decoder
    u1 = UpSampling2D((2, 2))(c5)
    u1 = Concatenate()([u1, c4])
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    c6 = BatchNormalization()(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', padding='same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Dropout(dropout_rate)(c6)
    
    u2 = UpSampling2D((2, 2))(c6)
    u2 = Concatenate()([u2, c3])
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    c7 = BatchNormalization()(c7)
    c7 = Conv2D(128, (3, 3), activation='relu', padding='same')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Dropout(dropout_rate)(c7)
    
    u3 = UpSampling2D((2, 2))(c7)
    u3 = Concatenate()([u3, c2])
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    c8 = BatchNormalization()(c8)
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Dropout(dropout_rate)(c8)
    
    u4 = UpSampling2D((2, 2))(c8)
    u4 = Concatenate()([u4, c1])
    c9 = Conv2D(32, (3, 3), activation='relu', padding='same')(u4)
    c9 = BatchNormalization()(c9)
    c9 = Conv2D(32, (3, 3), activation='relu', padding='same')(c9)
    c9 = BatchNormalization()(c9)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    model = Model(inputs, outputs)
    return model

2025-06-10 14:05:49.754116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749564349.774477 3253954 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749564349.780488 3253954 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749564349.797906 3253954 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749564349.797943 3253954 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749564349.797945 3253954 computation_placer.cc:177] computation placer alr

Ergebnisse:
- das Model scheint nicht viel Besser als das Originale zu sein

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.utils import Sequence

# # def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
# def unet_model(input_size, dropout_rate=0.2):
#   inputs = Input(input_size)

#   # Encoder
#   c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
#   c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
#   p1 = MaxPooling2D((2, 2))(c1)
#   p1 = Dropout(dropout_rate)(p1)

#   c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
#   c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
#   p2 = MaxPooling2D((2, 2))(c2)

#   c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
#   c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(c3)
#   p3 = MaxPooling2D((2, 2))(c3)

#   # Bottleneck
#   c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
#   c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

#   # Decoder
#   u1 = UpSampling2D((2, 2))(c4)
#   u1 = Concatenate()([u1, c3])
#   c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
#   c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

#   u2 = UpSampling2D((2, 2))(c5)
#   u2 = Concatenate()([u2, c2])
#   c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
#   c6 = Conv2D(32, (3, 3), activation='relu', padding='same')(c6)

#   u3 = UpSampling2D((2, 2))(c6)
#   u3 = Concatenate()([u3, c1])
#   c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
#   c7 = Conv2D(16, (3, 3), activation='relu', padding='same')(c7)

#   outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)

#   model = Model(inputs, outputs)
#   return model